In [ ]:
#!git clone "https://github.com/jyanivaddi/ERA_V1.git"
#!git pull

Add all the imports

In [1]:
from __future__ import print_function
import sys
sys.path.append("ERA_V1/session_7")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from s8_utils import load_cifar10_data, preview_batch_images, plot_statistics
from s8_model import Model_Net, model_summary, model_train, model_test

Allocate GPU

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
cifar_10_model = Model_Net().to(device)
model_summary(cifar_10_model, input_size=(3,32,32))

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [2, 1, 3, 32, 32]

Define Transforms

In [ ]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.244, 0.262))
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.244, 0.262))
])

Download Data

In [ ]:
train_data, test_data, class_names = load_cifar10_data(train_transforms, test_transforms)

Define train and test loaders

In [ ]:
torch.manual_seed(1)
batch_size = 128
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = True, **kwargs)

Calculate Mean and Std of dataset

In [ ]:
from tqdm import tqdm
import numpy as np
mean_r = 0
mean_g = 0
mean_b = 0

std_r = 0
std_g = 0
std_b = 0
for img,_ in tqdm(train_loader):
    img = np.asarray(img)
    #print(img[:,0,:,:].shape)
    num_elem = np.prod(img[:,0,:,:].shape)
    mean_r += np.mean(img[:,0,:,:])
    mean_g += np.mean(img[:,1,:,:])
    mean_b += np.mean(img[:,2,:,:])
    std_r += np.var(img[:,0,:,:])
    std_g += np.var(img[:,1,:,:])
    std_b += np.var(img[:,2,:,:])
mean_r = mean_r / len(train_loader)
mean_g = mean_g / len(train_loader)
mean_b = mean_b / len(train_loader)
std_r = np.sqrt(((1./(len(train_loader)-1))*std_r))
std_g = np.sqrt(((1./(len(train_loader)-1))*std_g))
std_b = np.sqrt(((1./(len(train_loader)-1))*std_b))
print(f"dataset mean: {mean_r:0.3f}, {mean_g:0.3f}, {mean_b:0.3f}")
print(f"dataset std: {std_r:0.3f}, {std_g:0.3f}, {std_b:0.3f}")


Show sample Images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def preview_images(train_loader, num_rows = 5, num_cols = 5):
    batch_data, batch_label = next(iter(train_loader))
    num_images_to_preview = num_rows*num_cols
    fig = plt.figure()
    for cnt in range(num_images_to_preview):
        plt.subplot(num_rows,num_cols,cnt+1)
        plt.tight_layout()
        this_img = np.asarray(batch_data[cnt])
        plt.imshow(this_img.transpose((1,2,0)))
        plt.title(class_names[str(batch_label[cnt].item())])
        plt.xticks([])
        plt.yticks([])
    plt.show()

preview_images(train_loader,num_rows = 5, num_cols = 5)

In [ ]:
model = Model_Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1, verbose=True)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True)
train_losses = []
test_losses = []
train_acc = []
test_acc = []
for epoch in range(1,20):
    print(f"epoch: {epoch}")
    this_train_loss = model_train(model, device, train_loader, optimizer, train_acc, train_losses)
    this_loss = model_test(model, device, test_loader, test_acc, test_losses)
    #scheduler.step()
    #print(optimizer.)


Plot Statistics

In [ ]:
plot_statistics(train_losses, train_acc, test_losses, test_acc)